Labels are as follows:

0: Joyful   
1: Upset  

In [ ]:
# Imports

import os
import csv
import cv2
import glob
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
from random import randint

import tensorflow as tf
import seaborn as sns
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from skimage.transform import resize
from tensorflow import keras
from tensorflow.keras import Input, layers
from tensorflow.keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras.models import Model, Sequential, load_model
from keras.layers import Conv2D, MaxPool2D, MaxPooling2D
from keras.layers.core import Activation, Dropout, Flatten, Dense
from keras.utils.vis_utils import plot_model
from keras import backend as K
from keras import layers
from sklearn.preprocessing import OneHotEncoder
from tqdm import tqdm

%matplotlib inline

In [ ]:
# Instantiate Blank Arrays

joyful_paths, joyful_images = [], []
upset_paths, upset_images = [], []

In [ ]:
file_name = "../input/mood-recognition/dataset/train.csv"

with open(file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=",")
    line_ct = 0

    for row in csv_reader:
        if(line_ct >= 1):
            val_to_check = row[1]
            if(val_to_check == 'joyful'): joyful_paths.append(row[0])
            if(val_to_check == 'upset'): upset_paths.append(row[0])
                
        line_ct += 1

joyful_labels = [0] * len(joyful_paths)   
upset_labels = [1] * len(upset_paths)          

In [ ]:
for joy_path in tqdm(joyful_paths):
    image = cv2.imread('../input/mood-recognition/dataset/train/' + joy_path)
    image = cv2.resize(image, dsize=(200,200))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    joyful_images.append(image)

for ups_path in tqdm(upset_paths):
    image = cv2.imread('../input/mood-recognition/dataset/train/' + ups_path)
    image = cv2.resize(image, dsize=(200,200))
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    upset_images.append(image)

In [ ]:
joyful_images_2, joyful_labels_2 = np.array(joyful_images[5731:]), np.array(joyful_labels[5731:])
joyful_images, joyful_labels = np.array(joyful_images[:5731]), np.array(joyful_labels[:5731])

upset_images_2, upset_labels_2 = np.array(upset_images[3950:]), np.array(upset_labels[3950:])
upset_images, upset_labels = np.array(upset_images[:3950]), np.array(upset_labels[:3950])

X_train = np.append(joyful_images, upset_images, axis = 0)
y_train = np.append(joyful_labels, upset_labels)
X_test = np.append(joyful_images_2, upset_images_2, axis = 0)
y_test = np.append(joyful_labels_2, upset_labels_2)

In [ ]:
# Checking if images + labels show correctly

fig, axes = plt.subplots(ncols=10, nrows=3, figsize=(16, 4))

indices = np.random.choice(len(X_train), 30)
counter = 0

for i in range(3):
    for j in range(10):
        axes[i,j].set_title(y_train[indices[counter]])
        axes[i,j].imshow(X_train[indices[counter]], cmap='gray')
        axes[i,j].get_xaxis().set_visible(False)
        axes[i,j].get_yaxis().set_visible(False)
        counter += 1
plt.show()

In [ ]:
# Reshaping X_train and y_train

one_hot_encoder = OneHotEncoder(sparse=False)

y_train = y_train[:, np.newaxis]
y_train_one_hot = one_hot_encoder.fit_transform(y_train)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], X_train.shape[2], 1)

In [ ]:
# Reshaping X_test and y_test

one_hot_encoder = OneHotEncoder(sparse=False)

y_test = y_test[:, np.newaxis]
y_test_one_hot = one_hot_encoder.fit_transform(y_test)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], X_test.shape[2], 1)

In [ ]:
# Basic CNN

input_shape = (X_train.shape[1], X_train.shape[2], 1)
input1 = Input(shape=input_shape)

cnn = Conv2D(16, (3, 3), activation='relu', strides=(1, 1), 
    padding='same')(input1)
cnn = Conv2D(32, (3, 3), activation='relu', strides=(1, 1), 
    padding='same')(cnn)
cnn = MaxPool2D((2, 2))(cnn)

cnn = Conv2D(16, (2, 2), activation='relu', strides=(1, 1), 
    padding='same')(cnn)
cnn = Conv2D(32, (2, 2), activation='relu', strides=(1, 1), 
    padding='same')(cnn)
cnn = MaxPool2D((2, 2))(cnn)

cnn = Flatten()(cnn)
cnn = Dense(100, activation='relu')(cnn)
cnn = Dense(50, activation='relu')(cnn)
output1 = Dense(2, activation='softmax')(cnn)

model = Model(inputs=input1, outputs=output1)

model.summary()

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

plot_model(model, show_shapes=True, to_file='mood_cnn.png')

In [ ]:
y_train = to_categorical(y_train, 2)
y_test = to_categorical(y_test, 2)

history = model.fit(x = X_train, y = y_train, validation_data = (X_test, y_test), epochs = 50)

In [ ]:
model.save("./mood_checker.h5")

In [ ]:
plt.plot(history.history['accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['train'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['train'], loc='upper right')
plt.show()

In [ ]:
!zip -r mood_check.zip '/kaggle/working/mood_checker.h5'